In [13]:
import numpy as np
import cv2
import pandas as pd
import os
from skimage.metrics import structural_similarity as ssim

In [16]:
# Ground Truth Path
path_to_ground_truth = 'output_images/0_trunc/'

# Truncation Values
truncation_values = [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50]

# Make a DataFrame to store the results. Rows are image names, columns are truncation values
results = pd.DataFrame(columns=truncation_values)


In [3]:
def load_pgm_opencv(file_path):
    # Load PGM image using OpenCV
    image = cv2.imread(file_path, cv2.IMREAD_UNCHANGED)
    return image

In [5]:
# Treat the original image as the ground truth and the approximated image as the prediction.
# Calculate the F1 score.

def calculate_f1_score(original_image, approximated_image):
    # Calculate the F1 score
    true_positive = np.sum(np.logical_and(original_image, approximated_image))
    false_positive = np.sum(np.logical_and(np.logical_not(original_image), approximated_image))
    false_negative = np.sum(np.logical_and(original_image, np.logical_not(approximated_image)))

    precision = true_positive / (true_positive + false_positive)
    recall = true_positive / (true_positive + false_negative)

    f1_score = 2 * (precision * recall) / (precision + recall)
    return f1_score

In [17]:
# For every image in the ground truth folder, get the corresponding approximated image from output_images/{truncation_value}_trunc folder.
# Calculate the F1 score and store it in the results DataFrame.

for truncation_value in truncation_values:
    path_to_approximated_images = f'output_images/{truncation_value}_trunc/'
    for image in os.listdir(path_to_ground_truth):
        if image.endswith('.pgm'):
            original_image = load_pgm_opencv(path_to_ground_truth + image)
            approximated_image = load_pgm_opencv(path_to_approximated_images + image)
            ssim_score = ssim(original_image, approximated_image)
            results.loc[image, truncation_value] = ssim_score

In [18]:
results.head()

,0,5,10,15,20,25,30,35,40,45,50
181091.pgm,1.0,0.914131,0.825784,0.750035,0.679967,0.610584,0.554807,0.505205,0.461486,0.418249,0.375648
302003.pgm,1.0,0.948265,0.904148,0.864201,0.825288,0.774647,0.733985,0.700063,0.665989,0.639136,0.615725
35070.pgm,1.0,0.982869,0.956663,0.932102,0.899477,0.868101,0.841577,0.815345,0.796692,0.768751,0.757586
216066.pgm,1.0,0.91289,0.825792,0.745937,0.668292,0.596156,0.528855,0.462882,0.405342,0.354301,0.305052
166081.pgm,1.0,0.9283,0.856217,0.793161,0.730775,0.674354,0.624132,0.570448,0.528372,0.485041,0.446398


In [19]:
results.to_csv('logs/susan_logs.csv')

In [20]:
# Get the average of each column
average_ssim = results.mean()

print(average_ssim)

0          1.0
5     0.940115
10    0.877902
15    0.822536
20    0.765141
25    0.710787
30    0.663364
35    0.615986
40    0.572202
45    0.535664
50    0.499644
dtype: object


In [ ]:
# Generate a csv file that contains 30000 values of 3.6 

dummy_volts = np.full((30000, 1), 3.6)
dummy_volts_df = pd.DataFrame(dummy_volts, columns=['Voltage'])
dummy_volts_df.to_csv('logs/dummy_volts.csv', index=False)

In [15]:
# Ground Truth Path
path_to_ground_truth = 'output_images/original/'

# Prediction Path
path_to_prediction = 'output_images/truncation_45perc/'

# Path to Logs

path_to_logs = 'logs/f1_truncation_45perc.csv'

# For every image in the ground truth folder, calculate the F1 score with the corresponding image in the prediction folder if the image exists. 
# The log file contains the name of all images in the first column, another field in the second column. The F1 score should get appended in the third column.

import os
import pandas as pd

ground_truth_images = os.listdir(path_to_ground_truth)
prediction_images = os.listdir(path_to_prediction)

log_data = []

for image in ground_truth_images:
    if image in prediction_images:
        original_image = load_pgm_opencv(path_to_ground_truth + image)
        approximated_image = load_pgm_opencv(path_to_prediction + image)
        f1_score = calculate_f1_score(original_image, approximated_image)
        log_data.append([image, f1_score])
    else:
        log_data.append([image, 'N/A'])
        
log_df = pd.DataFrame(log_data, columns=['Image', 'F1 Score'])
log_df.to_csv(path_to_logs, index=False)

